<a href="https://colab.research.google.com/github/Huy-SPKT-k16/CNN/blob/main/Breast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        641M         11G        1.1M        1.1G         11G
Swap:            0B          0B          0B


In [2]:
# !cd "/content/drive/MyDrive/Colab Notebooks"
# !unzip -u "/content/drive/MyDrive/Colab Notebooks/Benign.zip" -d "../root/.keras/datasets/BreastDataset/"
# !unzip -u "/content/drive/MyDrive/Colab Notebooks/Invasive.zip" -d "../root/.keras/datasets/BreastDataset/"
# !unzip -u "/content/drive/MyDrive/Colab Notebooks/Normal.zip" -d "../root/.keras/datasets/BreastDataset/"

In [28]:
import numpy as np 
import matplotlib.pyplot as plt 
import pathlib
import os, sys
import cv2 
import pickle
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/dataset"
data_dir = pathlib.Path(data_dir)
categories = os.listdir(data_dir) 
number_classes = len(categories)

In [5]:
batch_size = 32
img_height = 1536
img_width = 2048
scale_percent = 25
width = int(img_width * scale_percent / 100)
height = int(img_height * scale_percent / 100)
dim = (width, height)

In [6]:
X = []
y = []

In [7]:
# new_img_array = None

# def create_training_data():
#   for category in categories: 
#     path = os.path.join(data_dir, category)  
#     label_number = categories.index(category) 
#     for img in os.listdir(path): 
#     # try:
#       img_array     = cv2.imread(os.path.join(path, img), cv2.IMREAD_ANYCOLOR) 
#       new_img_array = cv2.resize(img_array, dim, interpolation = cv2.INTER_AREA) 
#       new_img_array = np.array(new_img_array)

#       X.append(new_img_array)  
#       y.append(label_number)  
#     # except Exception as e:
#     #   pass
#     print(category + " success!")

# create_training_data()

In [8]:
# # Chạy 1 lần, thành công thì không chạy nữa
# pickle_out = open("/content/drive/MyDrive/Colab Notebooks/X.pickle", "wb")
# pickle.dump(X, pickle_out)
# pickle_out.close()

In [9]:
# # Chạy 1 lần, thành công thì không chạy nữa
# pickle_out = open("/content/drive/MyDrive/Colab Notebooks/y.pickle", "wb")
# pickle.dump(y, pickle_out)
# pickle_out.close()

In [10]:
# Mỗi lần chạy mô hình thì chạy để load dữ liệu
pickle_in = open("/content/drive/MyDrive/Colab Notebooks/X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("/content/drive/MyDrive/Colab Notebooks/y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [11]:
X = np.array(X)

In [12]:
X = X / 255

In [13]:
X.shape

(1119, 384, 512, 3)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X = y = []
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=0.2, random_state=42)
y_train = to_categorical(y_train) # one hot
# ví dụ y_number = 1 thì y = [0. 1. 0.]
y_t = y_test
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [23]:
model = Sequential() 

model.add(Conv2D(filters=16, kernel_size=(3, 3), input_shape=X_train.shape[1:], padding="same", activation="relu"))
model.add(AveragePooling2D(pool_size=(4, 4), strides=2))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="tanh"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="selu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="selu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(number_classes))
model.add(Activation("softmax"))

# Cung cấp cho mô hình hàm loss function lựa chọn, thuật toán tối ưu
# categorical_crossentropy, Binary Cross Entropy
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary() # xem chi tiết mô hình xây dựng của chính mình

# Cuối cùng đào tạo nó
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=25)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 384, 512, 16)      448       
_________________________________________________________________
average_pooling2d_7 (Average (None, 191, 255, 16)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 189, 253, 32)      4640      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 94, 126, 32)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 92, 124, 64)       18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 46, 62, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 44, 60, 64)       

In [24]:
cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [25]:
mkdir -p saved_model

In [26]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [29]:
new_model = load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 384, 512, 16)      448       
_________________________________________________________________
average_pooling2d_7 (Average (None, 191, 255, 16)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 189, 253, 32)      4640      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 94, 126, 32)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 92, 124, 64)       18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 46, 62, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 44, 60, 64)       

In [31]:
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

7/7 - 13s - loss: 0.6794 - accuracy: 0.7768
Restored model, accuracy: 77.68%
